# Exercise: Create a BERT sentiment classifier

In this exercise, you will create a BERT sentiment classifier using the [Hugging Face Transformers](https://huggingface.co/transformers/) library. You will use the [IMDB movie review dataset](https://ai.stanford.edu/~amaas/data/sentiment/) to train and evaluate your model.

The IMDB dataset contains 50,000 movie reviews that are labeled as either positive or negative. The dataset is split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.


In [ ]:
# Install dependencies for this notebook if they are missing
! pip install -q \
    scikit-learn \
    evaluate \
    datasets \
    "transformers[torch]" \
    ipywidgets

In [ ]:
# Since we intend on using a GPU, we inspect that one is available and has enough memory to run the model
! nvidia-smi

In [ ]:
# Import the datasets and transformers packages
# NOTE: If you receieve an error such as "ModuleNotFoundError: No module named 'datasets'",
# please restart the kernel (Kernel > Restart) and start the notebook from the top

from datasets import load_dataset

from transformers import AutoTokenizer

# Load the imdb dataset: https://huggingface.co/datasets/imdb
# Take only the "train" and "test" splits
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits,load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(100))

# Show the first training example
# dataset["train"][0]
# dataset["train"]
ds

In [ ]:

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(lambda x: tokenizer(x["text"], truncation=True), batched=True)
print(tokenized_ds["train"][0]["input_ids"])

# batch_size = 32

# train_loader = DataLoader(train_dataset.with_format(type='torch'), batch_size=32)

# for batch_idx, (data, label) in enumerate(train_loader):
#     print(batch_idx, data, label)
#     if batch_idx > 10:
#         break

# train_dataset.with_format("torch")


In [ ]:

import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


import numpy as np

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}


# https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

# Freeze all the parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
print(model)

In [ ]:
# Check which layers are frozen
# for param in model.base_model.parameters():
#     print(param.requires_grad)
# len(list(model.base_model.parameters()))
# print the nubmer of layers

# Show the weights of the first 5 layers

old = [model.base_model.embeddings.word_embeddings.weight.data[0,:10],
model.base_model.transformer.layer[0].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[1].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[2].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[3].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[4].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[5].attention.q_lin.weight.data[0,:10],
model.pre_classifier.weight.data[0,:10],
model.classifier.weight.data[0,:10],
]
print(old)


In [ ]:
from transformers import DataCollatorWithPadding

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer 
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,

    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

new = [model.base_model.embeddings.word_embeddings.weight.data[0,:10],
model.base_model.transformer.layer[0].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[1].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[2].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[3].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[4].attention.q_lin.weight.data[0,:10],
model.base_model.transformer.layer[5].attention.q_lin.weight.data[0,:10],
model.pre_classifier.weight.data[0,:10],
model.classifier.weight.data[0,:10],
]

In [ ]:
print(old)

In [ ]:
print(new)

In [ ]:
import torch
for old_item, new_item in zip(old, new):
    # move to cpu device
    old_item = old_item.cpu()
    new_item = new_item.cpu()

    print((torch.eq(old_item, new_item)))


In [ ]:
print(model.base_model.embeddings.word_embeddings.weight.data[0,:10])
print(old_values)

In [ ]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()


In [ ]:
import pandas as pd
df = pd.DataFrame(tokenized_ds["test"])
df = df[['text', 'label']]
df = pd.concat(
    [
        df[df['label'] == 0].head(2),
        df[df['label'] == 1].head(2)
    ]
)
df

In [ ]:
# show entire cell in pandas
pd.set_option('display.max_colwidth', 200)
df

In [ ]:
dataset_indices = list(df.index)

In [ ]:
# Show the performance of the model on some test set examples

# select the first negative and the first positive test set examples


predictions = trainer.predict(tokenized_ds["test"].select(dataset_indices))
# Print the 10 first test set samples with their predictions.
# split text into lines of 80 characters
def split_text(text, n=160):
    lines = []
    while len(text) > n:
        line = text[:n]
        space_index = line.rfind(" ")
        if space_index != -1:
            line = line[:space_index]
        lines.append(line)
        text = text[len(line):]
    lines.append(text)
    return "\n".join(lines)

for index, (pred, label, text) in enumerate(zip(predictions.predictions.argmax(axis=1), predictions.label_ids, tokenized_ds["test"].select(dataset_indices)["text"])):
    print(f"{index}. pred={id2label[pred]} | label={id2label[label]} \n {split_text(text)}")